### 문제정의
- 네이버 영화리뷰 데이터셋을 이용해서 긍정/부정 분류기(모델)를 만들자.
- TF-IDF 방법을 적용해 보자
- konlpy 한국어 형태소 분석기를 설치하고 활용해 보자
- 단어별 긍정/부정 정보를 시각화해보자

In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 38.1 MB/s eta 0:00:00


In [2]:
from konlpy.tag import Okt

In [3]:
okt = Okt()

In [4]:
okt.morphs('아버지가방에들어가신다')
# morphs : 가지고 있는 문장을 명사단위로 쪼개주는 기능
# 한글은 띄어쓰기 단위로 토큰화를 진행하게 될 경우 의미가 손실
# 토큰화를 할 수 있는 모듈을 따로 사용해야 함

['아버지', '가방', '에', '들어가신다']

konlpy 간단하게 사용해보기

In [5]:
text = ["우리반 오늘도 화이팅.",
        "우리반 내일도 화이팅.",
        "우리반 고생했어요.",
        "우리반 최고."]

Countvectorizer(BOW) : 토큰화

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
# 모델생성
CV = CountVectorizer()

In [8]:
# 학습
# 토큰화, 단어사전 구축
CV.fit(text)
# 토큰화 기본값 - 띄어쓰기 단위로 토큰화

CountVectorizer()

In [10]:
CV.vocabulary_

{'우리반': 3, '오늘도': 2, '화이팅': 5, '내일도': 1, '고생했어요': 0, '최고': 4}

Countvectorizer + Okt

In [11]:
# 함수 생성 (명사단위)
def myToken(text) :
  return okt.nouns(text)  # 명사단위로 추출해서 토큰화

In [12]:
# 모델 생성
cv_okt = CountVectorizer(tokenizer=myToken)

In [13]:
# 학습
cv_okt.fit(text)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(tokenizer=<function myToken at 0x7ad2fb7c7d90>)

In [14]:
cv_okt.vocabulary_

{'우리': 4, '반': 2, '오늘': 3, '화이팅': 6, '내일': 1, '고생': 0, '최고': 5}

In [15]:
# 토큰화 도구 : morphs
# okt.morphs ==> 형태소별로 구분
okt.morphs(text[0])

['우리', '반', '오늘', '도', '화이팅', '.']

In [16]:
okt.pos(text[0])
# okt.pos : 토큰화 한 후 어떤 형태소를 가지고 있는지 확인해주는 함수

[('우리', 'Noun'),
 ('반', 'Noun'),
 ('오늘', 'Noun'),
 ('도', 'Josa'),
 ('화이팅', 'Noun'),
 ('.', 'Punctuation')]

In [ ]:
# okt가 가지고 있는 전체 형태소 확인
okt.tagset

In [ ]:
from konlpy.tag import Kkma
kkma = Kkma()
kkma.tagset
# 데이터 전처리시 시간적 여유가 많다면 사용
# okt에 비해 더 많은 형태소로 구분 ==> 시간이 많이걸림

네이버 영화리뷰 데이터셋 감성분석

In [21]:
import numpy as np
import pandas as pd

데이터 불러오기

In [22]:
text_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML/data/ratings_train.csv', encoding='utf-8')
text_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML/data/ratings_test.csv', encoding='utf-8')

In [ ]:
text_train

In [ ]:
text_test

In [25]:
# 결측치 확인
text_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [26]:
text_test.info()
# document에 결측치 ==> 부정인지 긍정인지 평가만하고 문자평은 남기지 않은 데이터

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [27]:
# 결측치가 있는 행을 삭제
text_train.dropna(inplace=True)   # 5
text_test.dropna(inplace=True)    # 3

In [29]:
text_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [30]:
# 문제만 토큰화시키기 위해서 문제와 정답으로 나누기
X_train = text_train['document']
X_test = text_test['document']
y_train = text_train['label']
y_test = text_test['label']

In [31]:
# TF-IDF 적용
# 문장의 빈도(가중치, 페널티 적용)

토큰화 및 수치화 --> pipeline

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [33]:
# pipeline 으로 연결
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

In [34]:
pipe_model = make_pipeline(TfidfVectorizer(tokenizer=myToken),
                           LogisticRegression())

In [ ]:
# 학습
pipe_model.fit(X_train, y_train)

In [36]:
# 평가
pipe_model.score(X_train, y_train)

0.8072535751191706

In [37]:
pipe_model.score(X_test, y_test)

0.768846130767846

In [38]:
# pipe_model --> TF-IDF(단어사전), LogisticRegression(가중치 - coef_)

tfidf = pipe_model.steps[0][1] # 단어사전
logi = pipe_model.steps[1][1]  # 가중치
# steps : 단어사전에서 하나씩 꺼내온다

In [39]:
word_weights = logi.coef_[0]  # 2차원에서 하나의 데이터를 꺼내오기
voca = tfidf.vocabulary_

In [40]:
# 단어들을 인덱스 순서대로 정렬
df = pd.DataFrame([voca.keys(), voca.values()])
# 번호를 인덱스로 사용하기 위해 전치, 행과 열을 뒤집음
df = df.T
# 1번 컬럼을 기준으로 정렬
df = df.sort_values(by=1)
# 가중치(coef) 컬럼 추가
df['coef'] = word_weights
# 가중치를 기준으로 내림차순 정렬
df = df.sort_values(by='coef', ascending=False)
df

,0,1,coef
175,최고,31950,6.632190
567,여운,22493,5.613019
141,수작,18299,5.453489
1146,최고다,31952,5.040493
2119,꿀잼,4505,4.833556
...,...,...,...
310,짜증,30747,-5.584751
1786,차라리,31095,-5.591531
388,실망,19690,-5.596437
168,쓰레기,20116,-6.990575


In [41]:
# 긍정에 영향을 주는 단어 확인(10)
df.head(10)

,0,1,coef
175,최고,31950,6.632190
567,여운,22493,5.613019
141,수작,18299,5.453489
1146,최고다,31952,5.040493
2119,꿀잼,4505,4.833556
466,명작,11107,4.332856
3235,굿굿,2901,3.992591
428,강추,629,3.976017
274,가슴,132,3.931742
64,짱,30798,3.925484


In [42]:
# 부정에 영향을 주는 단어 확인(10)
df.tail(10)

,0,1,coef
1226,망작,10459,-4.369554
211,돈,7498,-4.676507
137,별로,14042,-4.862154
371,졸작,29414,-5.306630
89,노잼,5666,-5.445845
310,짜증,30747,-5.584751
1786,차라리,31095,-5.591531
388,실망,19690,-5.596437
168,쓰레기,20116,-6.990575
418,최악,32018,-9.822472
